In [11]:
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="admin:password123"))
client.heartbeat()  # this should work with or without authentication - it is a public endpoint
client.get_or_create_collection("test_collection")  # this is a protected endpoint and requires authentication
client.list_collections()  # this is a protected endpoint and requires authentication

[Collection(name=test_collection3),
 Collection(name=test_collection1),
 Collection(name=test_collection2),
 Collection(name=test_collection)]

In [13]:
col = client.get_or_create_collection("test_collection")
col.add(ids=["1"],documents=["test doc"])

In [14]:
col.delete(ids=["1"])

In [28]:
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="user1:password1223"))
client.heartbeat()  # this should work with or without authentication - it is a public endpoint
client.get_or_create_collection("test_collection")  # this is a protected endpoint and requires authentication
client.list_collections()  # this is a protected endpoint and requires authentication

AuthorizationError: Unauthorized

In [13]:
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="invalid_user:password123"))
client.heartbeat()  # this should work with or without authentication - it is a public endpoint

client.list_collections()  # this is a protected endpoint and requires authentication

AuthorizationError: Unauthorized

# Testing Authorization

## Scenario 1 - Admin user


In [28]:
import uuid
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="admin:password123"))
client.heartbeat()  # this should work with or without authentication - it is a public endpoint
client.list_collections()  # this is a protected endpoint and requires authentication

col = client.get_or_create_collection(f"test_collection-{str(uuid.uuid4())}")
col.add(ids=["1"],documents=["test doc"])

col.get()
col.update(ids=["1"],documents=["test doc 2"])
col.count()
col.upsert(ids=["1"],documents=["test doc 3"])
col.delete(ids=["1"])

client.delete_collection(col.name)


## Scenario 2 - Reader user


In [57]:
import uuid
import chromadb
from chromadb.config import Settings

client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="admin:password123"))
client.heartbeat()  # this should work with or without authentication - it is a public endpoint
client.list_collections()  # this is a protected endpoint and requires authentication

col_name = f"test_collection-{str(uuid.uuid4())}"
col = client.get_or_create_collection(col_name)
print(f"Creating collection {col.id}" )
col.add(ids=["1"],documents=["test doc"])

client.get_collection(col_name)
client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="user1:password123"))

client.heartbeat()  # this should work with or without authentication - it is a public endpoint
client.list_collections()  # this is a protected endpoint and requires authentication
client.count_collections()
print("Getting collection " + col_name)
col = client.get_collection(col_name)
col.get()
col.count()


try:
    client.delete_collection(col_name)
except Exception as e:
    print(e) #expect unauthorized error

client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="admin:password123"))

client.delete_collection(col_name)

Creating collection d07ccf65-ac38-4ef8-981a-f4a27558fd92
Getting collection test_collection-0e8c3bfc-8830-4933-898b-7ea1bd356fab
Unauthorized


## Scenario 3 - Cross user access

In [2]:
import uuid
import chromadb
from chromadb.config import Settings

col_name = f"test_collection-{str(uuid.uuid4())}"
client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="admin:password123"))

client.get_or_create_collection(col_name)

client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="admin-ext:password123"))

client.get_or_create_collection("external-collection")

try:
    client.delete_collection(col_name)
except Exception as e:
    print("Expected error for admin-ext: ",str(e)) #expect unauthorized error

client = chromadb.HttpClient(
    settings=Settings(chroma_client_auth_provider="chromadb.auth.basic.BasicAuthClientProvider",chroma_client_auth_credentials="admin:password123"))
client.delete_collection(col_name)
try:
    client.delete_collection("external-collection")
except Exception as e:
    print("Expected error for admin: ",str(e)) #expect unauthorized error

Expected error for admin-ext:  Unauthorized
Expected error for admin:  Unauthorized
